In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer,
)
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score,precision_score,f1_score,recall_score
import pickle
import wandb
np.random.seed(42)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
X = data['Date']
y  = data['btc_cost_per_transaction']

In [4]:
new_X = []

In [5]:
for x in X.tolist():
    try:
        x = x.split('-')
        x = int(f'{x[0]}{x[1]}{x[2]}')
        new_X.append(x)
    except:
        x = x[0].split('/')
        x = int(f'{x[0]}{x[1]}{x[2]}')
        new_X.append(x)

In [6]:
X = np.array(new_X)

In [7]:
y = np.array(y.tolist())

In [8]:
from flaml import AutoML

In [9]:
def valid(model,X,y,val=False):
    y_test = y
    y_preds = model.predict(X)
    try:
            results = {
                'Accuracy':model.score(X,y),
                'MAE':mean_absolute_error(y_test,y_preds),
                'MSE':mean_squared_error(y_test,y_preds),
            }
    except:
            results = {
                'MAE':mean_absolute_error(y_test,y_preds),
                'MSE':mean_squared_error(y_test,y_preds),
            }
    return results
def fit(model,X_train,X_test,y_train,y_test,name):
    wandb.init(project=PROJECT_NAME,name=name)
    model = model
    try:
        model.fit(X_train,y_train,task='classification')
    except:
        model.fit(X_train,y_train)
    print('Logging')
    wandb.log(valid(model,X_train,y_train))
    wandb.log(valid(model,X_test,y_test,True))
    print('Saving')
    pickle.dump(model,open(f'./models/model-{name}.pkl','wb'))
    print('Making Submissions')
    make_submission(model)
    return model

In [11]:
pres = [StandardScaler(),RobustScaler(),MinMaxScaler(),MaxAbsScaler(),Normalizer(),LabelEncoder()]

In [ ]:

models = [
    ['KNeighborsRegressor',KNeighborsRegressor],
    ['DecisionTreeRegressor',DecisionTreeRegressor],
    ['GradientBoostingRegressor',GradientBoostingRegressor],
    ['AdaBoostRegressor',AdaBoostRegressor],
    ['RandomForestRegressor',RandomForestRegressor],
    ['BaggingRegressor',BaggingRegressor],
    ['SVR',SVR],
    ['XGBClassifier',XGBClassifier],
    ['XGBRFClassifier',XGBRFClassifier],
    ['CatBoostClassifier',CatBoostClassifier],
    ['CatBoost',CatBoost],
    ['FloatML',AutoML()]
]